In [17]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import torch.nn.functional as F

In [19]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')

In [20]:
def to_img(x):
    x = x.view(x.size(0),1,28,28)
    #print (x)
    return x

In [21]:
num_epochs = 100
batch_size = 128
learning_rate = 0.001

In [22]:
def plot_sample_img(img, name):
    img=img.view(1,28,28)
    save_img(img, './sample_{}.png'.format(name))

In [23]:
def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor/max_tensor
    tensor = tensor*(max_value - min_value) + min_value
    return torch.round(tensor)

In [24]:
img_transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0,1))])

In [25]:
dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle= True)

In [26]:
'''class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28,256),
            nn.ReLU(True),
            nn.Linear(256,64),
            nn.ReLU(True))
        
        self.decoder = nn.Sequential(
            nn.Linear(64,256),
            nn.ReLU(True),
            nn.Linear(256,28*28),
            nn.Sigmoid())
        # Define forward
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
'''
#Convolution autoencoder
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784,400)
        self.fc21 = nn.Linear(400,20)
        
        self.fc22 = nn.Linear(400,20)
        self.fc3 = nn.Linear(20,400)
        self.fc4 = nn.Linear(400,784)
        
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return(eps.mul(std).add_(mu))
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar
    
        

In [27]:
model = VAE().cuda()
model

VAE(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
)

In [28]:
reconstruction_function = nn.MSELoss(reduction='sum')

In [29]:
def loss_function(recon_x,x,mu,logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x,x) #mse loss
    KLD_element = mu.pow(2).add(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return BCE + KLD
    

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [31]:
# Training

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for batch_idx, data in enumerate(dataloader):
        img,_=data
        img = img.view(img.size(0),-1)
        img = Variable(img)
        img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        
        optimizer.step()
        
        if batch_idx%100 ==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.data[0] / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')
            
        
    

        

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [0/60000 (0%)]	Loss: 200.068222
Train Epoch: 0 [12800/60000 (21%)]	Loss: 60.820198
Train Epoch: 0 [25600/60000 (43%)]	Loss: 56.998211
Train Epoch: 0 [38400/60000 (64%)]	Loss: 53.264763
Train Epoch: 0 [51200/60000 (85%)]	Loss: 48.470757
====> Epoch: 0 Average loss: 59.1955
Train Epoch: 1 [0/60000 (0%)]	Loss: 49.527145
Train Epoch: 1 [12800/60000 (21%)]	Loss: 46.030945
Train Epoch: 1 [25600/60000 (43%)]	Loss: 44.787491
Train Epoch: 1 [38400/60000 (64%)]	Loss: 45.672977
Train Epoch: 1 [51200/60000 (85%)]	Loss: 49.175491
====> Epoch: 1 Average loss: 46.9194
Train Epoch: 2 [0/60000 (0%)]	Loss: 45.553169
Train Epoch: 2 [12800/60000 (21%)]	Loss: 45.926346
Train Epoch: 2 [25600/60000 (43%)]	Loss: 46.085052
Train Epoch: 2 [38400/60000 (64%)]	Loss: 44.592369
Train Epoch: 2 [51200/60000 (85%)]	Loss: 43.382977
====> Epoch: 2 Average loss: 44.5986
Train Epoch: 3 [0/60000 (0%)]	Loss: 44.127457
Train Epoch: 3 [12800/60000 (21%)]	Loss: 44.664948
Train Epoch: 3 [25600/60000 (43%)]	Loss: 

Train Epoch: 28 [12800/60000 (21%)]	Loss: 40.843956
Train Epoch: 28 [25600/60000 (43%)]	Loss: 42.929707
Train Epoch: 28 [38400/60000 (64%)]	Loss: 38.067795
Train Epoch: 28 [51200/60000 (85%)]	Loss: 39.324825
====> Epoch: 28 Average loss: 40.1380
Train Epoch: 29 [0/60000 (0%)]	Loss: 40.101845
Train Epoch: 29 [12800/60000 (21%)]	Loss: 39.571793
Train Epoch: 29 [25600/60000 (43%)]	Loss: 39.458401
Train Epoch: 29 [38400/60000 (64%)]	Loss: 41.017693
Train Epoch: 29 [51200/60000 (85%)]	Loss: 40.413425
====> Epoch: 29 Average loss: 40.0772
Train Epoch: 30 [0/60000 (0%)]	Loss: 40.807220
Train Epoch: 30 [12800/60000 (21%)]	Loss: 39.909081
Train Epoch: 30 [25600/60000 (43%)]	Loss: 40.375530
Train Epoch: 30 [38400/60000 (64%)]	Loss: 41.292297
Train Epoch: 30 [51200/60000 (85%)]	Loss: 40.549934
====> Epoch: 30 Average loss: 40.0348
Train Epoch: 31 [0/60000 (0%)]	Loss: 41.132996
Train Epoch: 31 [12800/60000 (21%)]	Loss: 41.125149
Train Epoch: 31 [25600/60000 (43%)]	Loss: 40.904026
Train Epoch: 31 [

Train Epoch: 56 [0/60000 (0%)]	Loss: 38.417976
Train Epoch: 56 [12800/60000 (21%)]	Loss: 40.100990
Train Epoch: 56 [25600/60000 (43%)]	Loss: 39.600861
Train Epoch: 56 [38400/60000 (64%)]	Loss: 39.766800
Train Epoch: 56 [51200/60000 (85%)]	Loss: 40.314648
====> Epoch: 56 Average loss: 39.3092
Train Epoch: 57 [0/60000 (0%)]	Loss: 38.618359
Train Epoch: 57 [12800/60000 (21%)]	Loss: 40.449371
Train Epoch: 57 [25600/60000 (43%)]	Loss: 39.953636
Train Epoch: 57 [38400/60000 (64%)]	Loss: 38.853462
Train Epoch: 57 [51200/60000 (85%)]	Loss: 39.693199
====> Epoch: 57 Average loss: 39.2978
Train Epoch: 58 [0/60000 (0%)]	Loss: 38.647041
Train Epoch: 58 [12800/60000 (21%)]	Loss: 39.303291
Train Epoch: 58 [25600/60000 (43%)]	Loss: 39.012657
Train Epoch: 58 [38400/60000 (64%)]	Loss: 38.761372
Train Epoch: 58 [51200/60000 (85%)]	Loss: 39.124977
====> Epoch: 58 Average loss: 39.2842
Train Epoch: 59 [0/60000 (0%)]	Loss: 39.844719
Train Epoch: 59 [12800/60000 (21%)]	Loss: 38.316460
Train Epoch: 59 [25600

====> Epoch: 83 Average loss: 38.8891
Train Epoch: 84 [0/60000 (0%)]	Loss: 38.848965
Train Epoch: 84 [12800/60000 (21%)]	Loss: 38.761551
Train Epoch: 84 [25600/60000 (43%)]	Loss: 38.929626
Train Epoch: 84 [38400/60000 (64%)]	Loss: 38.470779
Train Epoch: 84 [51200/60000 (85%)]	Loss: 39.719677
====> Epoch: 84 Average loss: 38.8344
Train Epoch: 85 [0/60000 (0%)]	Loss: 38.293709
Train Epoch: 85 [12800/60000 (21%)]	Loss: 38.353153
Train Epoch: 85 [25600/60000 (43%)]	Loss: 38.786720
Train Epoch: 85 [38400/60000 (64%)]	Loss: 40.660889
Train Epoch: 85 [51200/60000 (85%)]	Loss: 39.202862
====> Epoch: 85 Average loss: 38.8259
Train Epoch: 86 [0/60000 (0%)]	Loss: 37.454071
Train Epoch: 86 [12800/60000 (21%)]	Loss: 38.912170
Train Epoch: 86 [25600/60000 (43%)]	Loss: 39.492912
Train Epoch: 86 [38400/60000 (64%)]	Loss: 40.029083
Train Epoch: 86 [51200/60000 (85%)]	Loss: 39.418365
====> Epoch: 86 Average loss: 38.8172
Train Epoch: 87 [0/60000 (0%)]	Loss: 40.613663
Train Epoch: 87 [12800/60000 (21%)]	

In [32]:
os.getcwd()

'D:\\1st year\\Python codes\\CIFR Analysis\\self contained analysis'